In [1]:
# pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.5.1+cu118.html
# pip install fastapi uvicorn
# pip install sqlalchemy

In [17]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool, GATConv
from torch_geometric.data import Data
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
class EnhancedGNNLSTMModel(nn.Module):
    def __init__(self, node_feature_dim, edge_feature_dim, hidden_dim, output_dim):
        super(EnhancedGNNLSTMModel, self).__init__()
        
        # GNN Layers
        self.gnn1 = GATConv(node_feature_dim, hidden_dim)
        self.gnn2 = GATConv(hidden_dim, hidden_dim)
        self.gnn3 = GCNConv(hidden_dim, hidden_dim)
        
        # Graph Pooling
        self.pool = global_mean_pool
        
        # LSTM Layers
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=2, bidirectional=True, batch_first=True)
        
        # Attention Mechanism
        self.attention = nn.Linear(hidden_dim * 2, 1)
        
        # Final Output Layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # GNN Forward Pass
        x = self.gnn1(x, edge_index)
        x = torch.relu(x)
        x = self.gnn2(x, edge_index)
        x = torch.relu(x)
        x = self.gnn3(x, edge_index)
        x = torch.relu(x)

        # Pooling
        x = self.pool(x, batch)

        # Reshape for LSTM (batch, sequence_length, feature_dim)
        x = x.unsqueeze(1)

        # LSTM Forward Pass
        lstm_out, _ = self.lstm(x)

        # Attention Mechanism
        attn_weights = torch.softmax(self.attention(lstm_out).squeeze(-1), dim=1)
        context = torch.bmm(attn_weights.unsqueeze(1), lstm_out).squeeze(1)

        # Output Layer
        out = self.fc(context)
        return out

# Example Data for Testing
def create_example_data():
    node_features = torch.rand((9, 3))  # 5 nodes, 3 features each
    edge_index = torch.tensor([[0, 1, 2, 3, 4, 0],
                                [1, 2, 3, 4, 0, 2]])  # 6 edges
    edge_attr = torch.rand((6, 2))  # 6 edges, 2 features each
    batch = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])  # Single graph in batch

    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, batch=batch)
    return data

# Model and Example
node_feature_dim = 3
edge_feature_dim = 2
hidden_dim = 16
output_dim = 1

model = EnhancedGNNLSTMModel(node_feature_dim, edge_feature_dim, hidden_dim, output_dim)
data = create_example_data()

# Forward Pass
output = model(data)
out = torch.sigmoid(output)
print("Sigmoid Output:", out)


Sigmoid Output: tensor([[0.5100]], grad_fn=<SigmoidBackward0>)


In [2]:
from fastapi import FastAPI

app = FastAPI()

@app.get('/')
def 작명():
    return 'hello world'

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite (from versions: none)
ERROR: No matching distribution found for sqlite
